In [1]:
import pandas as pd
import datetime as dt
from datetime import date,time,datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
lbl = LabelEncoder()
mlb = MultiLabelBinarizer()
import numpy as np
%matplotlib inline
from sklearn.externals import joblib

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
df = pd.read_pickle("../dataset.pkl")

In [3]:
df

,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,50808,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
0,2,0,65,462.0,0.0,24.0,22.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,1,1,0
1,6,1,71,462.0,0.0,24.0,26.0,1.0,104.0,1.20,...,0,0,0,0,0,0,0,0,0,1
2,7,1,75,462.0,1.0,24.0,27.0,1.0,98.0,1.07,...,0,0,0,0,0,1,0,0,0,1
3,3,1,39,462.0,0.0,24.0,27.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,0,0,0
4,4,1,59,462.0,0.0,24.0,26.0,1.0,104.0,1.11,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,3,0,20,230.0,-2.0,26.0,25.0,1.0,101.0,1.12,...,0,0,0,0,0,0,0,1,1,0
58972,1,0,84,485.0,-3.0,18.0,18.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,0,0,1
58973,9,1,42,422.0,-1.0,24.0,27.0,1.0,103.0,1.08,...,0,1,0,1,0,0,0,1,1,1
58974,11,0,60,462.0,-3.0,22.0,23.0,2.0,111.0,1.00,...,0,0,0,0,0,0,0,1,1,0


In [4]:
x = df.iloc[:,:483]
x.shape
#483 = 3(adm_days,gender,age)+480(lab values)

(58976, 483)

In [5]:
y = df.iloc[:,483:]


In [6]:
y.columns

Index(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')

In [7]:
from sklearn.model_selection import train_test_split # seed?
seed=243 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15, random_state=seed)

In [8]:
y.shape

(58976, 20)

In [9]:
x_train.shape

(50129, 483)

In [10]:
y_train.shape

(50129, 20)

In [11]:
x_test.shape

(8847, 483)

In [12]:
y_test.shape

(8847, 20)

In [13]:
np.unique(y_train)

array([0, 1])

In [14]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
sample_weights = [1-3032/58976,1-8924/58976,1-35608/58976,1-19366/58976,
                 1-16271/58976,1-20008/58976,1-43243/58976,
                 1-25281/58976,1-21200/58976,1-21782/58976,1-166/58976,          
                 1-8316/58976,1-10053/58976,1-3205/58976,
                 1-5492/58976,1-19454/58976,1-1757/58976,1-23352/58976,1-16621/58976,1-34350/58976]
weight_rf = [{0:3032/58976,1:1-3032/58976},{0:8924/58976,1:1-8924/58976},{0:1-35608/58976,1:35608/58976},{0:19366/58976,1:1-19366/58976},
            {0:16271/58976,1:1-16271/58976},{0:20008/58976,1:1-20008/58976},{0:1-43243/58976,1:43243/58976},
            {0:25281/58976,1:1-25281/58976},{0:21200/58976,1:1-21200/58976},{0:21782/58976,1:1-21782/58976},{0:166/58976,1:1-166/58976},
            {0:8316/58976,1:1-8316/58976},{0:10053/58976,1:1-10053/58976},{0:3205/58976,1:1-3205/58976},
            {0:5492/58976,1:1-5492/58976},{0:19454/58976,1:1-19454/58976},{0:1757/58976,1:1-1757/58976},
            {0:23352/58976,1:1-23352/58976},{0:16621/58976,1:1-16621/58976},{0:1-34350/58976,1:34350/58976}]*58976
weights = {0:0.286,1:0.714}#this for catboost
#[0,1] classes 0.286 is probabbility of 1 in dataset(icd) so i flipped it over as to give more weight to 1

#model1 = CatBoostClassifier(iterations=1000, task_type="GPU",devices='0:1',learning_rate=0.1)
#d_train = lgb.Dataset(x_train, label=y_train)
#clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=100))
clf_multilabel = OneVsRestClassifier(XGBClassifier(1500,n_jobs=-1,class_weight=weights))

In [15]:
'''parameters = {
    
    
    "learning_rate":[0.1, 0.2, 0.3, 0.4]
}

model_tunning = GridSearchCV(clf_multilabel, param_grid=parameters,n_jobs=-1,cv=10,
                             scoring = 'roc_auc')

model_tunning.fit(x_train, y_train)

print(model_tunning.best_score_)
print(model_tunning.best_params_)'''

'parameters = {\n    \n    \n    "learning_rate":[0.1, 0.2, 0.3, 0.4]\n}\n\nmodel_tunning = GridSearchCV(clf_multilabel, param_grid=parameters,n_jobs=-1,cv=10,\n                             scoring = \'roc_auc\')\n\nmodel_tunning.fit(x_train, y_train)\n\nprint(model_tunning.best_score_)\nprint(model_tunning.best_params_)'

In [16]:
#estimator.get_params().keys()

In [17]:
cbc = clf_multilabel.fit(x_train,y_train)
joblib.dump(cbc, "XGB1-C.pkl")

['XGB1-C.pkl']

In [18]:
#print(clf_multilabel.best_score_)
mod = joblib.load("XGB1-C.pkl")

In [19]:
y_pred = mod.predict(x_test)

In [20]:
y_pred

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 1],
       ...,
       [0, 1, 1, ..., 1, 0, 1],
       [0, 0, 1, ..., 1, 0, 1],
       [0, 0, 1, ..., 0, 0, 0]])

In [21]:
#cm = confusion_matrix(y_test,y_pred) 
#Classification metrics can't handle a mix of multilabel-indicator and unknown targets
print(jaccard_score(y_test,y_pred,average='micro'))
print(jaccard_score(y_test,y_pred,average='macro'))
print(roc_auc_score(y_test,y_pred))
print(hamming_loss(y_test,y_pred))
#look at this lol
# RF ?yep look up ok go on


0.5060543826501274
0.369680282359682
0.6643581037392547
0.16991070419351192


In [22]:
'''#even train and test set changes everytime as there is random shuffling
#to take into account that as well i have transferred it to pickle
#run this before ml2
x_train.to_pickle('./x_train.pkl')
y_train.to_pickle('./y_train.pkl')
x_test.to_pickle('./x_test.pkl')
y_test.to_pickle('./y_test.pkl')'''

"#even train and test set changes everytime as there is random shuffling\n#to take into account that as well i have transferred it to pickle\n#run this before ml2\nx_train.to_pickle('./x_train.pkl')\ny_train.to_pickle('./y_train.pkl')\nx_test.to_pickle('./x_test.pkl')\ny_test.to_pickle('./y_test.pkl')"

In [23]:
#y_predtest_prob.to_pickle("./y_predtest_probcat.pkl")

In [24]:
#y_predtrain_prob.to_pickle("./y_predtrain_probcat.pkl")